# Projet de Maths 1

## Contexte

On se donne une fonction $f: \mathbb{R}^2 \to \mathbb{R}$ continument différentiable. Notre objectif est de créer une fonction qui permet de tracer un contour pour une valeur $c$ donnée.

## Amorce 

On suppose ici $f$ définie sur $[0,1]^2$. On veut tout d'abord trouver un point de départ sur le coté gauche du carré pour tracer notre contour. Pour cela, on cherche un $y \in [0,1]$ tel que $f(0,y)=c$. 
On procède par dichotomie et l'on impose donc à $f$ de vérifier : $c \in [f(0,0),f(0,1)] $ ou $c \in [f(0,1),f(0,0)]$ .

In [0]:
def find_seed(g,c=0,debut=0,fin=1,eps=2**(-26)):
    if g(debut)<=c<=g(fin):         # si c'est le premier cas qui est vérifié
        a=debut     
        b=fin
    elif g(fin)<=c<=g(debut):       # si c'est le deuxième c
        a=fin
        b=debut
    else :                          # si aucun des deux n'est vérifié, on ne peut pas trouver par dichotomie un tel y
        return None
    while b-a>eps:                  # on veut une précision de eps sur la valeur de y
        milieu=(a+b)/2              
        if c<=g(milieu):            # recherche par dichotomie
            b=milieu
        else :
            a=milieu
    return float((a+b)/2)

### Exemple

0n teste notre fonction sur $f : x \mapsto x^2 -2$. On va obtenir une valeur approchée à eps de $\sqrt{2}$.

In [5]:
def f_1(x):
    return x**2 -2

find_seed(f_1,0,1,2)

## Contour simple 

On peut maintenant, à partir de la fonction précédente qui nous donne un point de départ, trouver un fragment de ligne de niveau.
On procède de la manière suivante : à partir du point que l'on vient de trouver, on calcule le gradient de $f$ en ce point puis un vecteur normal au gradient de norme $\delta$, puis on se dirige dans la direction du vecteur normal au gradient (ou dans la direction opposée si l'on sort du carré). Ainsi, on obtient une liste de points espacés de $\delta$ qui sont censés vérifier $f(x,y)=c$.

In [8]:
import autograd
from autograd import numpy as np
import matplotlib.pyplot as plt

def simple_contour(f,c=0.0,delta=0.01,nb_point=100):

    def grad_f(x,y):                          # calcule le gradient de f à partir de autograd
        gr=autograd.grad
        return np.r_[gr(f,0)(x,y),gr(f,1)(x,y)]

    def g(t):                                # pour calculer le point de départ du contour sur le coté gauche
        return f(0,t)

    x=[0.0]                                    # point de départ
    y=[find_seed(g,c)]

    if y[0]==None:                   #si l'on n'a pas trouvé de point de départ
        return [],[]

    for i in range(nb_point):                 # on limte le nombre de points que l'on cherche pour avoir un programme qui finit
        grad=grad_f(x[i],y[i])                # on calcule le gradient au dernier point que l'on a trouvé
        aux=[grad[1],-grad[0]]                # puis un vecteur auxiliaire normal au gradient
        norme_aux=np.sqrt(aux[0]**2+aux[1]**2)  # puis sa norme pour le renormaliser
        vect_orthon=[delta/norme_aux*aux[0],delta/norme_aux*aux[1]]   # on calcule un vecteur normal au gradient de norme delta

        posx=x[i]+vect_orthon[0]      # abscisse du potentiel point suivant
        posy=y[i]+vect_orthon[1]      # ordonnée du potentiel point suivant


        if posx>1 or posx<0 or posy>1 or posy<0: # si jamais on sort du carré, on part dans l'autre direction
            posx=posx-2*vect_orthon[0]
            posy=posy-2*vect_orthon[1]
            
            if posx>1 or posx<0 or posy>1 or posy<0:  # si l'on sort encore, on s'arrête
                return x,y
       
        x.append(posx)    # on ajoute le nouveau point à la liste
        y.append(posy)
    
    return x,y    #on retourne le fragment de contour


### Test 

In [10]:
def f_test(x,y):
    return np.power(x,2)+np.power(y,2)

plt.close()
x,y=simple_contour(f_test,0.01)
plt.plot(x,y)
plt.show()

On contaste que le programme ne renvoie pas le résultat attendu ce qui est dû au fait que le vecteur noramal au gradient donne une direction dans laquelle il peut y avoir un point sur le contour mais c'est assez approximatif.
Pour ressoudre ce problème, on a essayé de calculer les points suivants en mettant une bouce while avec la condition d'arrêt : $\mid f(x,y)-c \mid \leq \frac{\mid c \mid}{100}$ pour ne pas trop s'éloigner de la ligne de niveau mais nous obtenions pas de résultats satisfaisants non plus car le programme n'effectuait qu'une dizaine d'étape dans le meilleur des cas.

Nous avons donc décider de changer de méthode et d'utiliser find_seed à chaque étape sur un segment du carré donné par le gradient. METTRE DESSIN !!!!

In [11]:
def simple_contour(f,c=0.0,delta=0.01,nb_point=100):

    def grad_f(x,y):                              # on commence de la même manière
        gr=autograd.grad
        return np.r_[gr(f,0)(x,y),gr(f,1)(x,y)]

    def g(t):
        return f(0,t)

    x=[0.0]
    y=[find_seed(g,c)]

    for i in range(nb_point):
        if x[i]==None or y[i]==None:         # si jamais on n'a pas pu trouver 
            return x[:i],y[:i]               # on renvoie les valeurs qu'on a deja trouvé
        grad=grad_f(x[i],y[i])
        aux=[grad[1],-grad[0]]
        norme_aux=np.sqrt(aux[0]**2+aux[1]**2)
        vect_orthon=[delta/norme_aux*aux[0],delta/norme_aux*aux[1]]
        posx=x[i]+vect_orthon[0]
        posy=y[i]+vect_orthon[1]
        if posx>1 or posx<0 or posy>1 or posy<0:
            posx=posx-2*vect_orthon[0]
            posy=posy-2*vect_orthon[1]
            if posx>1 or posx<0 or posy>1 or posy<0:
                return x,y

        if grad[0]==0.0:                               # si jamais le gradient est parallèle à l'axe des ordonnées 
            x.append(posx)
            y.append(find_seed(lambda t:f(posx,t),c,max(posy-delta,0),min(posy+delta,1)))

        else :                                         # dans le cas général
            t=find_seed(lambda t:f(t,posy+(t-posx)*grad[1]/grad[0]),c,max(posx-delta,0),min(posx+delta,1)) 
            x.append(t)
            try :                    # si jamais t=None, pour ne pas avoir une erreur o
                y.append(posy+(t-posx)*grad[1]/grad[0])      
            except:
                y.append(0)          # on met 0 par défaut et de toute façon, on va le supprimer à l'étape d'après
    return x,y

### Exemple

In [12]:
plt.close()
x,y=simple_contour(f_test,0.01)
plt.plot(x,y)
plt.show()

Résultat beaucoup plus satisfaisant !!